In [ ]:
import pandas as pd 
import numpy as np
import glob
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

In [2]:
import datetime

def xldate_to_datetime(xldate):
    temp = datetime.datetime(1900, 1, 1)
    delta = datetime.timedelta(days=xldate)
    return temp+delta

In [16]:
#which quantile bin does the days return fall into?
#plot a histogram of each bin
centroids = pd.read_csv('/Users/livingdharma/Downloads/ProgrammingTest/spxFwdRetHistQCentroids.csv')

In [18]:
rtns = pd.read_csv('/Users/livingdharma/Downloads/ProgrammingTest/SpxFwdPct_Data.csv')

In [264]:
#rtns = rtns.drop('Lookup',axis=1)
rtns = rtns.set_index(keys='date')

In [25]:
day_10_ma = pd.read_csv('/Users/livingdharma/Downloads/ProgrammingTest/10DAYMA.TXT')
day_10_ma['date'] = day_10_ma['Lookup'].apply(xldate_to_datetime)

In [251]:

fls1 = glob.glob('/Users/livingdharma/Downloads/ProgrammingTest/*.txt')
fls2= glob.glob('/Users/livingdharma/Downloads/ProgrammingTest/*.TXT')
fls_all = fls1[:1] + fls2[:5] + fls2[6:]

In [311]:
dfs = []
for fl in fls_all:
    df = pd.read_csv(fl)
    col_name = df.columns[1]
    df['date'] = df['Lookup'].apply(xldate_to_datetime)
    df = df.drop('Lookup', axis=1)
    df = df.set_index(keys='date')
    print(df.index.min(), col_name)
    dfs.append(df)
    
    
feature_matrix_3_16 = pd.concat([df for df in dfs], join='inner', axis=1)
feature_matrix_3_16 = feature_matrix_3_16.dropna()
cols = feature_matrix_3_16.columns

1990-03-16 00:00:00 SectorCorrBD126
1990-01-04 00:00:00 1000 - 2000 Sharpe
1990-01-04 00:00:00 Diff10DayMA
1990-01-04 00:00:00 % Away From 200d MA
1990-01-04 00:00:00 Vol Dist from 21d MA of Vol
1990-01-04 00:00:00 Lagged AD % vs. 10d MA
1990-01-04 00:00:00 21d MA of Breakdown
1990-01-04 00:00:00 EM - USA Sharpe


In [312]:
#run some ml stuff
#check the 'goodness' of the model
#normalize the columns, rerun and evaluate
#add in other columns and rerun, evaluate
#add in other columns normalize, rerun, evaluate
#feature engineer the columns
feature_matrix_3_16_wrtns = pd.concat([feature_matrix_3_16, rtns], join='inner', axis =1)
feature_matrix_3_16_wrtns = feature_matrix_3_16_wrtns.dropna(axis=0)
split = int(len(feature_matrix_3_16)*.8)
split

5337

In [323]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
cols = feature_matrix_3_16.columns
cols
X_train = feature_matrix_3_16[cols][:split]
X_test = feature_matrix_3_16[cols][split:]
y_train = feature_matrix_3_16_wrtns['3'][:split]
y_test = feature_matrix_3_16_wrtns['3'][split:]
lr_model = linear_model.LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [324]:
y_predict = lr_model.predict(X_test)

In [325]:
mean_squared_error(y_test, y_predict)

0.00018516112767483678